In [18]:
import random
import simpy

In [19]:
SEED = 9999
random.seed(SEED)

In [43]:
class Source(object):
    ''' Generate the customers comming into the system with some exponetially distributed interval time.
    '''
    def __init__(self, env, numCustomers, interval, serveTime, server):
        self.env = env
        self.numCustomers = numCustomers
        self.interval = interval
        self.serveTime = serveTime
        self.server = server
        self.action = env.process(self.run())
    
    def run(self):
        ''' Create n customers and timeout after each creation
        '''
        for i in range(1, self.numCustomers+1):
            customer = Customer(self.env, i, self.server, self.serveTime)
            timeUntillNext = random.expovariate(1. / self.interval)
            yield self.env.timeout(timeUntillNext)

class Customer(object):
    ''' Customers request the passed resource if they can make use of an open service.
        They are then served which takes some exponentially distributed time.
    '''
    def __init__(self, env, number, server, serveTime):
        self.env = env
        self.number = number
        self.server = server
        self.serveTime = serveTime
        self.action = env.process(self.run())
    
    def run(self):
        ''' Create a service request and use it once it becomes available
        '''
#         print('customer %d starts at %d' % (self.number, self.env.now))
        with self.server.request() as req:
            yield req
#             print('customer %d served at %d' % (self.number, self.env.now))
            yield self.env.timeout(random.expovariate(1./self.serveTime))
            results.append(self.env.now)
#             print('customer %d done at %d' % (self.number, self.env.now))

results = []
CUSTOMERS = 100
CAPACITY = 1
MU = 10.0
LAMBDA = 2.
# MU = LAMBDA / CAPACITY + .1
env = simpy.Environment()
server = simpy.Resource(env, capacity=1)
source = Source(env, CUSTOMERS, LAMBDA, MU, server)
env.run()
print(results)

[2.7966731698279106, 5.37992942034267, 10.659481559594823, 13.711045715048787, 21.209908230260893, 24.21982391180446, 31.626543003130486, 52.384295966162995, 56.8023722003412, 80.93243821434581, 113.46677299625217, 117.56521698753157, 139.46667794606358, 144.76747384864785, 153.0039488358299, 157.78877977437492, 170.6134229415906, 221.06036021364017, 222.29615110388727, 229.47575405395415, 231.1379623359863, 248.56896263962665, 252.1762820266164, 259.26366116750523, 260.029763265384, 264.2607077009419, 278.15774986305826, 281.13074386497243, 290.115869554717, 292.32455945116743, 303.1985790820127, 320.392490095831, 327.65193981018, 357.8740169740446, 361.28185049064996, 363.3032473658215, 389.8549996487274, 408.0415145919971, 415.60868714165235, 415.8769441190212, 423.7220013705354, 426.1602194644506, 491.43589417193994, 511.20709910259325, 514.7974047001484, 515.9086888349761, 532.0857824330366, 535.2194745325161, 544.615945777245, 548.424102445488, 551.9838801075294, 564.591529119053